## 准备数据

In [47]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    # normalize
    
    x = x.reshape(-1, 784).astype('float32') / 255.0  # 展平并归一化
    x_test = x_test.reshape(-1, 784).astype('float32') / 255.0  # 展平并归一化
    # print(x.shape, x_test.shape)
    return (x, y), (x_test, y_test)

In [4]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [27]:
class myModel:
    def __init__(self, input_dim=784, hidden_dim=128, output_dim=10):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([input_dim, hidden_dim], stddev=0.01))
        self.W2 = tf.Variable(tf.random.normal([hidden_dim, output_dim], stddev=0.01))
        self.b1 = tf.Variable(tf.zeros([hidden_dim]))
        self.b2 = tf.Variable(tf.zeros([output_dim]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        " 隐藏层 激活层 "
        h = tf.matmul(x, self.W1) + self.b1
        h = tf.nn.relu(h) # 激活函数
        logits = tf.matmul(h, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [44]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        # print(logits.shape)
        loss = compute_loss(logits, y)
    
    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [ ]:
train_data, test_data = mnist_dataset()

for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))
" 原本精确度: 0.846, 不对啊 原本从0.83开始训练的 "
" 我这边训练很多轮才达到 0.84 呜呜呜 "
print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 0.71185076 ; accuracy 0.82805
epoch 1 : loss 0.71131927 ; accuracy 0.8281
epoch 2 : loss 0.7107889 ; accuracy 0.82815
epoch 3 : loss 0.71025985 ; accuracy 0.82823336
epoch 4 : loss 0.7097319 ; accuracy 0.8283
epoch 5 : loss 0.7092053 ; accuracy 0.82836664
epoch 6 : loss 0.7086798 ; accuracy 0.82845
epoch 7 : loss 0.70815545 ; accuracy 0.82848334
epoch 8 : loss 0.7076324 ; accuracy 0.82858336
epoch 9 : loss 0.7071105 ; accuracy 0.8286667
epoch 10 : loss 0.70658976 ; accuracy 0.82878333
epoch 11 : loss 0.7060702 ; accuracy 0.82888335
epoch 12 : loss 0.70555186 ; accuracy 0.82888335
epoch 13 : loss 0.7050347 ; accuracy 0.8289667
epoch 14 : loss 0.7045187 ; accuracy 0.8290333
epoch 15 : loss 0.7040039 ; accuracy 0.82913333
epoch 16 : loss 0.7034902 ; accuracy 0.82926667
epoch 17 : loss 0.7029776 ; accuracy 0.82935
epoch 18 : loss 0.7024662 ; accuracy 0.8294333
epoch 19 : loss 0.70195603 ; accuracy 0.8294333
epoch 20 : loss 0.701447 ; accuracy 0.82945
epoch 21 : loss 0.700939